Run this notebook to

- (Apply a low-frequency filter to 0.5Hz and a high-pass filter to 30Hz
- Create feedback epochs (300ms before to 1s after feedback feedback, baselined at 100ms before to time of feedback feedback)
- Drop epochs that fall within EEG parts annotated as bad
- Set behavioural meta-data
- Drop epochs that exceed peak-to-peak amplitude range of 200 µV 
- Sub-select task block data (ie not practice trials)
- Save each participant's feedback epochs files
- Plot epoch timecourse for all channels
- Plot electrode Cz trace with stacked ERP plot sorted by reaction times
- Plot electrode Cz trace for joint_correct vs disjoint_correct trials
- Calculate z-scored confidence reports within each feedback condition and plot Cz trace for above vs below median confidence trials
- Check how many epochs remain per participant




To do 
- sanity check line-up of metadata with EEG data by plotting (scatterplot) diff in trigger times (feedback-feedback) against reaction times
- do the tacked ERP sorted by reaction times plot with vertical smoothing across epochs (currently only figured out how to horizonal smoothing, i.e. temporal within each epoch) to maybe bring out the diagonal red line more with P3 starting later for longer reaction times

In [ ]:
import os
import numpy as np
import mne
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,corrmap)
import matplotlib.pyplot as plt
#import seaborn as sns
import re
import pandas as pd

million = 1000000.


#%matplotlib qt
%matplotlib inline

input_dir = 'EEGdata/cleaned_mastoid_reference'



def mkdir(p):
    sp = re.split('/|\\\\', p)
    bp = ''
    for pp in sp:
        bp = os.path.join(bp, pp)
        if not os.path.exists(bp):
            os.mkdir(bp)
            print( '%s created.' % bp)

            
mkdir('TaskfeedbackEpochsMastoids')
mkdir('Figures/TaskfeedbackEpochsMastoids')
mkdir('Figures/TaskfeedbackEpochsMastoids/all channels')
mkdir('Figures/TaskfeedbackEpochsMastoids/correct vs incorrect')
mkdir('Figures/TaskfeedbackEpochsMastoids/z-scored confidence median split')
mkdir('Figures/TaskfeedbackEpochsMastoids/ERP by reaction time')



def plot_joint(erp, times, title='', width=12, height=8, invert=True, save=None):
    fig = erp.plot_joint(times=times,
                         show=False,
                         ts_args=dict(time_unit='s'),
                         topomap_args=dict(res=128, contours=4, time_unit='s'),
                         title=title)
    fig.set_figwidth(width)
    fig.set_figheight(height)
    axes = fig.get_axes()
    ax0 = axes[0]
    if invert:
        ax0.invert_yaxis()
    ch = ax0.get_children()
    for c in ch:
        if type(c) == plt.Annotation:
            c.remove()
        if type(c) == plt.Line2D:
            c.set_linewidth(2.5)
            c.set_alpha(.75)
    leg_ax = axes[-2]
    leg_ax.get_children()[0].set_sizes([50.])
    leg_ax.set_aspect('equal')
    if save is not None:
        fig.savefig(save)
    fig.show()



sessions = [1, 2]

epoch_numbers = {}

for session in sessions:
    
    # only strategic condition has feedback
    if session == 1:
        participant_numbers = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
    if session == 2:
        participant_numbers = [2, 4, 6, 8, 10, 12, 16, 18, 20, 21]
    
    for subject in participant_numbers:
        print('>>>> Session %i' % session)
        print('>>>> Participant %i' % subject)
        
        if session == 1:
            raw = mne.io.read_raw_fif('%s/s%i-raw.fif' % (input_dir, subject)).load_data() # session 1
            annot_file = '%s/%i-annot.fif' % ('annotations', subject)

        if session == 2:
            raw = mne.io.read_raw_fif('%s/s%i_2-raw.fif' % (input_dir, subject)).load_data() # session 2
            annot_file = '%s/%i_2-annot.fif' % ('annotations', subject)


        annotations = mne.read_annotations(annot_file)
        raw = raw.set_annotations(annotations)
        
        raw = raw.filter(l_freq=0.5, h_freq=30)
        
        events = mne.find_events(raw, stim_channel='Status')
        feedback_event_dict = {'feedback_correct': 13,
                               'feedback_incorrect': 14
                              }
        feedback_epochs = mne.Epochs(raw, events, tmin=-0.3, tmax=1, picks=['eeg'], event_id=feedback_event_dict,
                                     reject_by_annotation=True, preload=True, baseline=(-0.1, 0))
        
        drop_log = [v[0]  if (len(v) > 0) else 'ok' for v in feedback_epochs.drop_log]
        drop_log = np.array(drop_log)
        drop_log = drop_log[drop_log != 'IGNORED']
        
        if len(feedback_epochs) != len(drop_log[drop_log != 'BAD_']):
            raise Exception('Numbers of epochs do not match')
        
        
        alldata = pd.read_csv("mergedData/allData.csv")
        data = alldata[alldata['participant']== subject]
        
        if session == 1:
            data = data[data['session']== 1] # session 1
        if session == 2:
            data = data[data['session']== 2] # session 2
            
#         # for participant 8, session 2, the first 24 feedback events are missing in the EEG data because I started saving too late by accident.
#         # dropping these 24 trials from the meta-data (they are only practice trials anyways)
#         if subject == 8 and session == 2:
#             lost_drop_log = ['lost'] * 24
#             lost_drop_log = np.array(lost_drop_log)
#             drop_log = np.concatenate( (lost_drop_log, drop_log ) )
          
        data = data[100:] #### no feedback epochs for practice trials without confidence slider
        
        data['droplog'] = drop_log
        data = data[data['droplog'] == 'ok']
        #data.index = range(len(data))
        
        if len(feedback_epochs) != len(data):
            raise Exception('Number of epochs do not match.\n Found %i EEG events and %i csv events' % (len(epochs['feedback']), len(data)))
        
        metadata = pd.DataFrame(data=data)
        feedback_epochs.metadata = metadata
        
        reject_criteria = dict(eeg=200e-6)
        feedback_epochs.drop_bad(reject=reject_criteria)
        
        task_feedback_epochs = feedback_epochs['block_count > 2']
        
        epoch_numbers.update({'%i_%i' % (subject, session): '%i' % len(task_feedback_epochs)})
        
        
        if session == 1:
            task_feedback_epochs.save('TaskfeedbackEpochsMastoids/%i_1-epo.fif' % (subject), overwrite=True) # session 1
        if session == 2:
            task_feedback_epochs.save('TaskfeedbackEpochsMastoids/%i_2-epo.fif' % (subject), overwrite=True) # session 2
            
            
            
        condition = task_feedback_epochs.metadata['condition'].unique()
        condition = condition[0]
        
        erp = task_feedback_epochs.copy().average()
        plot_joint(erp, [-0.3, 0, 0.3, 0.6], title='Participant %i - feedback-locked - %s' % (subject, condition));
        if session == 1:
            plt.savefig('Figures/TaskfeedbackEpochsMastoids/all channels/%i_1.png' % (subject)) # session 1
        if session == 2:
            plt.savefig('Figures/TaskfeedbackEpochsMastoids/all channels/%i_2.png' % (subject)) # session 2
            
            
        sort_order = np.argsort(task_feedback_epochs.metadata['decision_rt'])
        task_feedback_epochs.plot_image(order=sort_order, picks='Cz') # set sigma to 1 or so if you want temporal smoothing
        if session == 1:
            plt.savefig('Figures/TaskfeedbackEpochsMastoids/ERP by reaction time/%i_1.png' % (subject)) # session 1
        if session == 2:
            plt.savefig('Figures/TaskfeedbackEpochsMastoids/ERP by reaction time/%i_2.png' % (subject)) # session 2
            
              
        correct_epochs = task_feedback_epochs['joint_correct == True']
        correct_evoked = correct_epochs.average()
        incorrect_epochs = task_feedback_epochs['joint_correct == False']
        incorrect_evoked = incorrect_epochs.average()
        evokeds = dict(correct=correct_evoked,incorrect=incorrect_evoked)
        mne.viz.plot_compare_evokeds(evokeds, picks=['Cz'], invert_y=False)
        plt.title('feedback-locked | P%i | %s' % (subject, condition))
        if session == 1:
            plt.savefig('Figures/TaskfeedbackEpochsMastoids/correct vs incorrect/%i_1.png' % (subject)) # session 1
        if session == 2:
            plt.savefig('Figures/TaskfeedbackEpochsMastoids/correct vs incorrect/%i_2.png' % (subject)) # session 2
        
        
        zscore = lambda x: (x - x.mean()) / x.std()
        task_feedback_epochs.metadata['confidence_z_by_partner'] = task_feedback_epochs.metadata['participant_confidence'].groupby(task_feedback_epochs.metadata['partner']).transform(zscore)
        
        median_confidence = task_feedback_epochs.metadata['confidence_z_by_partner'].median()
        low_conf_epochs = task_feedback_epochs['confidence_z_by_partner <= %i' % median_confidence]
        high_conf_epochs = task_feedback_epochs['confidence_z_by_partner > %i' % median_confidence]
        low_conf_evoked = low_conf_epochs.average()
        high_conf_evoked = high_conf_epochs.average()
        evokeds = dict(high_confidence=high_conf_evoked, low_confidence=low_conf_evoked)
        mne.viz.plot_compare_evokeds(evokeds, picks=['Cz'], invert_y=False)
        plt.title('feedback-locked | P%i | %s' % (subject, condition))
        if session == 1:
            plt.savefig('Figures/TaskfeedbackEpochsMastoids/z-scored confidence median split/%i_1.png' % (subject))
        if session == 2:
            plt.savefig('Figures/TaskfeedbackEpochsMastoids/z-scored confidence median split/%i_2.png' % (subject))

In [ ]:
# 16_2 and 21_2 may be problematic - check with Nick
dropped_epochs = {k: 300 - int(v) for k, v in epoch_numbers.items()}
dropped_epochs